# Pandas basics for using `alchemlyb`

This how-to assumes only working knowlege of Python.
You will learn how to effectively use the [pandas](https://pandas.pydata.org/) Python library, on which [alchemlyb](https://alchemlyb.readthedocs.io/en/latest/) depends.
This knowledge will serve you well in using `alchemlyb`, but also generalizes well beyond it.

You will learn how to do the following:
1. Parsing a dataset into a `pandas.DataFrame`.
2. Subselecting components of a DataFrame.
3. Obtaining descriptive statistics.
4. Modifying a DataFrame.

## Parsing a dataset

We'll begin by choosing an existing dataset in [alchemtest](https://alchemtest.readthedocs.io/en/latest/).
`alchemtest` features sample datasets from a variety of software packages, and is used as the test set for `alchemlyb`.
You will need to [install alchemtest](https://alchemtest.readthedocs.io/en/latest/install.html) if it is not already present in your environment.

In [ ]:
from alchemtest.gmx import load_expanded_ensemble_case_2

In [ ]:
dataset = load_expanded_ensemble_case_2()
print(dataset.DESCR)

`alchemtest` datasets have a `DESCR` attribute that gives metadata details about the dataset. This is important for interpreting the data files themselves correctly. We see from this description that this dataset features 2 "legs" for its alchemical pathway, with 20 windows for the Coulomb change and 12 windows for the VDW change.

The `data` attribute gives us paths to the dataset files as they are installed on our machine; if you are running this notebook on your own machines, these paths will likely differ:

In [ ]:
print(dataset.data)

We will load only one of these files to get started. To do this, we will need to use a parser function from `alchemlyb`. Since this is a dataset generated with [Gromacs](http://www.gromacs.org/), we will use a parser specific to Gromacs file outputs.

In [ ]:
datafile = dataset.data['AllStates'][0]
datafile

In [ ]:
from alchemlyb.parsing import gmx

In [ ]:
dHdl = gmx.extract_dHdl(datafile, T=300)
dHdl

In [ ]:
type(dHdl)

The `extract_dHdl` parser gives us a `pandas.DataFrame`, in `alchemlyb` [standard form for a dHdl](https://alchemlyb.readthedocs.io/en/latest/parsing.html#dhdl-standard-form) dataset. We will use this DataFrame for the rest of the lesson.

`pandas` is a library that provides special data structures for doing fast numerical operations on tabular data. Internally, ``pandas`` uses [``numpy``](http://www.numpy.org/) to do the heavy lifting. We will see in this lesson how `pandas` affords us both the flexibility and structure needed to effectively deal with alchemical data.

## The anatomy of a DataFrame

We saw earlier that our DataFrame looks essentially like this:

In [ ]:
dHdl

It features 25001 rows, and 4 columns ('fep', 'coul', 'vdw', 'restraint').

Each row is labeled with an *index* value, which for this dataset has 5 components ('time', 'fep-lambda', 'coul-lambda', 'vdw-lambda', 'restraint-lambda'), giving the state of the system at which the row was sampled.
Each column then gives the $\frac{\partial H}{\partial \lambda}$ corresponding to each of the lambda values being varied.

## Working with columns and rows

A ``DataFrame`` allows us to get at individual components of our tabular data. We can get single columns like:

In [ ]:
dHdl['coul']

Or multiple columns with:

In [ ]:
dHdl[['coul', 'restraint']]

Slicing can be used to get back subsets of rows:

In [ ]:
dHdl[0:5]

Python indices are 0-based, meaning counting goes as 0, 1, 2, 3...; this means that the first row is row 0, the second row is row 1, etc. It's best to refer to row 0 as the "zeroth row" to avoid confusion.

This slice should be read as "get the 0th element up to and not including the 5th element". The "not including" is important, and the cause of much initial frustration. It does take some getting used to.

What if we want a single row?

In [ ]:
dHdl[1]

For a DataFrame, this is ambiguous, since a single value is interpreted as a column name. We can only get at rows by slicing at the top level:

In [ ]:
dHdl[1:2]

Or we could be more explicit and use `.iloc`, which allows one to select rows (and columns) using 0-based indexes as one would do with e.g. `numpy` arrays:

In [ ]:
# select row 1, assuming a 0-based index
dHdl.iloc[1]

Getting a single row in this way returns a `pandas.Series`:

In [ ]:
type(dHdl.iloc[1])

A `Series` is a 1-D column of values, having all the same datatype. Since each of the datatypes of our columns were floating-point numbers, we got a `Series` with dtype `float64` this time. If we had columns with, e.g. strings, then we'd get back dtype `object`, which is a catchall for ``pandas``.

We can also get the data in our ``Series`` as a raw ``numpy`` array:

In [ ]:
type(dHdl.iloc[1].values)

Pandas recently made its 1.0 release, making it a stable base on which to build functionality such as that found in `alchemlyb`. What's more, it's built on top of the venerable ``numpy`` array, which makes it possible to do fast numerical work in Python. A `Series` is basically a 1-D ``numpy`` array with the ability to select by labeled indices.

We can also use `iloc` to select out individual elements of the `DataFrame` by specifying a column index:

In [ ]:
dHdl.iloc[1, 2]

Notice that this gets us the `'vdw'` value from the Series we saw above.

Earlier we saw that we can select single columns from a `DataFrame`; doing this also returns a `Series` object:

In [ ]:
dHdl['coul']

The columns of a DataFrame are effectively a set of `Series` objects bound together with a common index. Each column can have a different `dtype`, but all elements within a column must be of a single `dtype`.

In [ ]:
dHdl['coul'].dtype

In [ ]:
dHdl.dtypes

In this case, all our columns consist of floats, so we don't have to think too much about `dtype`s here. This is also true in general when working with `DataFrame`s produced by `alchemlyb` parsers.

## Subsetting data

Beyond slicing on an index, we can use boolean indexing to subselect our data. Say we want only data for which the `coul` column is greater than 50?

In [ ]:
dHdl[dHdl['coul'] > 50]

There's no magic here; we get a boolean index directly from a comparison:

In [ ]:
gt_50 = dHdl['coul'] > 50
gt_50

And using this `Series` of bools will then give only the rows for which the `Series` had `True`:

In [ ]:
dHdl[gt_50]

This is the same behavior as ``numpy`` for arrays: most binary operators, such as ``+``, ``*``, ``>``, ``&``, work element-wise. With a single value on one side (such as ``50``), we get the result of the operation for each element.

A ``DataFrame`` is an *object*, and objects have **methods**. These are functions that are *part of* the object itself, often doing operations on the object's data. One of these is ``DataFrame.mean``:

In [ ]:
dHdl.mean()

We get back the mean value of each column as a single ``Series``. There's more like this:

In [ ]:
dHdl.max()

There's also ``DataFrame.describe``, which gives common descriptive statistics of the whole `DataFrame`:

In [ ]:
dHdl.describe()

This is, itself, a ``DataFrame``:

In [ ]:
dHdl.describe()['coul']

Documentation is a key part of Python's design. In the notebook, you can get a quick look at the docs for a given Python function or method with:

In [ ]:
dHdl.mean?

Or more generally (built-in Python behavior):

In [ ]:
help(dHdl.mean)

--------------
### Challenge: obtain the standard deviation of `coul` $\frac{\partial H}{\partial \lambda}$ for samples in which `vdw` $\frac{\partial H}{\partial \lambda}$ is greater than 20

One way we could do this is to first grab the ``"coul"`` column, then use a fancy index obtained from comparing the ``"vdw"`` column to ``20``. We can then call the ``std`` method of the resulting ``Series``:

In [ ]:
dHdl['coul'][dHdl['vdw'] > 20].std()

Note that this is a key part of the power of ``pandas`` objects: operations for subsetting and calculating descriptive statistics can often be stacked to great effect.

-----------------------